In [ ]:
from urllib.request import urlopen
import bs4
from urllib.parse import quote
import csv

start_num = int(input())
page_num = int(input())
all_article = []
for i in range(start_num,page_num+1): #1페이지에 문제가 있어 2페이지부터 시작해야함 ,40페이지씩 끊는것을 추천
    print(i)
    #매경 검색창에 기간과 기업명을 설정하고 검색한 결과의 페이지
    url = "http://find.mk.co.kr/new/search.php?pageNum="+str(i)+"&cat=&cat1=&media_eco=&pageSize=10&sub=all&dispFlag=OFF&page=news&s_kwd=%BB%EF%BC%BA%C0%FC%C0%DA&s_page=news&go_page=&ord=1&ord1=1&ord2=0&s_keyword=%BB%EF%BC%BA%C0%FC%C0%DA&period=p_direct&s_i_keyword=%BB%EF%BC%BA%C0%FC%C0%DA&s_author=&y1=2018&m1=01&d1=01&y2=2018&m2=12&d2=31&media%5B1%5D=00001&ord=1&area=ttbd"
    print(url)
    while True:
        #검색결과가 나왔다 안나왔다 할때가 있어서 검색결과가 나온페이지의 태그가 뜰때까지 예외처리로 반복
        try:
            html = urlopen(url)
            bs = bs4.BeautifulSoup(html.read(),"html.parser")
            td = bs.find("td",{"class":"title"})
            a = td.text
            
        except AttributeError:
            print("check")
            span = bs.findAll("span",{"class":"art_tit"})
            print(span)
            for i in span:
                a = i.find("a")
                link = a["href"]
                if 'opinion' or 'sport' in link: #오피니언과 스포츠 기사 제외
                    pass
                html = urlopen(link)
                bs = bs4.BeautifulSoup(html.read(),"html.parser")
                empty =[]
                try:
                    h1 = bs.find("h1",{"class":"top_title"})
                    empty.append(h1.text)#제목
                except AttributeError:
                    print("오류기사")
                    pass
            
                try:
                    li = bs.find("li",{"class":"lasttime"})
                    empty.append(li.text[5:15]) #날짜  

                except AttributeError:
                    try:
                        li = bs.find("li",{"class":"lasttime1"})
                        empty.append(li.text[5:15]) #날짜
                    except:
                        empty.append("오류기사")
                
                try:
                    div = bs.find("div",{"class":"art_txt"})
                    article = div.get_text().replace('\r',"").replace('\n','').replace('\t','')
                    empty.append(article) #기사
                except AttributeError:
                    empty.append("오류기사 전문")
                    print("오류기사")
                empty.append(link) #링크
                all_article.append(empty)
                if empty[1][0] != '2'or '[표]' in empty[0] or (empty[0] or empty[1]) =="오류기사":
                    all_article.pop()
                
                print(link)
            break
    


f = open('articles.csv','a',encoding='utf-8-sig')
wr = csv.writer(f)
for i in all_article:
    if i[0] == i[2][5:len(i[0])+5]: #본문에 제목이 포함돼있는경우 그부분 제거
        i[2] = i[2][5+len(i[0]):]
    wr.writerow(i)

f.close()